In [5]:
import pickle
import numpy
from PIL import Image
import IPython.display
import os

import tracking.Tester
import tracking.RecurrentTracker
import tracking.TheanoGruRnn
import tracking.VideoSequence as vs
import tracking.VisualAttention
import tracking.GaussianGenerator

In [6]:
cd ../tracking

/home/fmpaezri/repos/localization-agent/tracking


In [7]:
with open('/home/fmpaezri/repos/localization-agent/model.pkl0', 'r') as modelFile:
    %time rnn = pickle.load(modelFile)

CPU times: user 10.3 s, sys: 768 ms, total: 11 s
Wall time: 13.5 s


In [8]:
tracker = tracking.RecurrentTracker.RecurrentTracker(None, rnn)

In [9]:
tester = tracking.Tester.Tester(tracker)

In [4]:
samplesDir = '/tmp/sampleSequences'

gg = tracking.GaussianGenerator.GaussianGenerator(
    '/home/fmpaezri/datasets/mscoco/',
    '/home/fmpaezri/repos/localization-agent/summaries/cocoTrainSummaryCategAndSideGt100SmplsAllCorrected.pkl',
    ['random', 'sine', 'stretched'], ['stretched'], None, grayscale=False, 
    objectPathTemplate='./train2014', scenePathTemplate='./train2014'
        )

%time gg.saveBatch(4, 0, 60, )

In [10]:
#sequencePath = '/home/fmpaezri/datasets/vot-challenge/sequences2014/basketball/'
sequencePath = os.path.join(samplesDir, numpy.random.choice(os.listdir(samplesDir)))

In [11]:
%time data, labels = tester.loadImageSequence(sequencePath, newShape=(rnn.imgSize,rnn.imgSize))

CPU times: user 28 ms, sys: 4 ms, total: 32 ms
Wall time: 33.7 ms


In [7]:
#ONLY FOR VOT BOXES
#Correcting box ccords as expected by IoU
#labelsCopy = labels.copy()
#labelsCopy[:,0] = labels[:,0::2].min(axis=1) 
#labelsCopy[:,1] = labels[:,1::2].min(axis=1)
#labelsCopy[:,2] = labels[:,0::2].max(axis=1) 
#labelsCopy[:,3] = labels[:,1::2].max(axis=1)
#labels = labelsCopy

In [13]:
data = data[numpy.newaxis, ...].astype(numpy.float32)

In [14]:
labels = labels[numpy.newaxis,...]

In [15]:
print data.shape, labels.shape

(1, 60, 100, 100, 3) (1, 60, 4)


In [16]:
%time iou, predLabels = tester.test(data, labels, None, rnn.batchSize, rnn.imgSize, False, labels.shape[1], 4, '/tmp')

CPU times: user 4.73 s, sys: 124 ms, total: 4.86 s
Wall time: 2.54 s


In [17]:
iou.mean()

0.080201329927709814

In [18]:
seq = vs.VideoSequence((Image.fromarray(frame.astype(numpy.uint8)) for frame in data[0]))

In [19]:
seq.addBoxes(labels[0], 'green')
seq.addBoxes((predLabels[0]), 'red')

In [20]:
outputName = '/tmp/testSequence1.mp4'
seq.exportToVideo(30, outputName)

In [21]:
IPython.display.HTML(vs.displayHTML(outputName))